In [ ]:
# !pip uninstall SentencePiece
!pip uninstall SentencePiece
!pip install transformers
!pip install pyvi 

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !pip3 install torch==1.10.1+cu113 torchvision==0.11.2+cu113 torchaudio===0.10.1+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html

In [2]:
from transformers import (
    T5Tokenizer,
    T5ForConditionalGeneration
)

In [ ]:
# Đưa câu tiếng việt vào
orig_sent = input("Nhập vào câu gốc: ")

In [5]:
from load_model1 import load_model_1

In [ ]:
import torch
if torch.cuda.is_available():
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")
device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: NVIDIA GeForce GTX 750


In [ ]:
path = '/content/drive/MyDrive/Vin_NLP_dataset/Inference_seq2seq'

In [ ]:
# Load module dịch vi-en
translation_model = T5ForConditionalGeneration.from_pretrained("NlpHUST/t5-vi-en-base")
translation_tokenizer = T5Tokenizer.from_pretrained("NlpHUST/t5-vi-en-base")
translation_model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(250112, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(250112, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedGeluDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (w

In [ ]:
# Load mô hình en-paraphrase
para_model = T5ForConditionalGeneration.from_pretrained('ramsrigouthamg/t5_paraphraser')
para_tokenizer = T5Tokenizer.from_pretrained('ramsrigouthamg/t5_paraphraser')
para_model = para_model.to(device)

In [ ]:
# Load mô hình dịch en-vi
# en_vi_model = T5ForConditionalGeneration.from_pretrained("NlpHUST/t5-en-vi-base")
en_vi_tokenizer = T5Tokenizer.from_pretrained("NlpHUST/t5-en-vi-base")
# en_vi_model.to(device)

RuntimeError: Internal: C:\ci\sentencepiece_1635181348757\work\src\sentencepiece_processor.cc(849) [model_proto->ParseFromArray(serialized.data(), serialized.size())] 

In [ ]:
!git lfs install
!git clone https://huggingface.co/NlpHUST/t5-en-vi-base
# if you want to clone without large files – just their pointers
# prepend your git clone with the following env var:
GIT_LFS_SKIP_SMUDGE=1


Updated git hooks.
Git LFS initialized.


In [ ]:
import sentencepiece as spm
spm._swig_python_version_info

sys.version_info(major=3, minor=6, micro=13, releaselevel='final', serial=0)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("NlpHUST/t5-en-vi-base")

model = AutoModelForSeq2SeqLM.from_pretrained("NlpHUST/t5-en-vi-base")

RuntimeError: Internal: C:\projects\sentencepiece\src\sentencepiece_processor.cc(891) [model_proto->ParseFromArray(serialized.data(), serialized.size())] 

In [ ]:
# Translate câu sang tiếng anh
def translation_vi_en(sentence):
    tokenized_text = translation_tokenizer.encode(sentence, return_tensors="pt").to(device)
    translation_model.eval()
    summary_ids = translation_model.generate(
                        tokenized_text,
                        max_length=256, 
                        num_beams=5,
                        repetition_penalty=2.5, 
                        length_penalty=1.0, 
                        early_stopping=True
                    )
    output = translation_tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    # print(output)
    return output

In [ ]:
# vi_en_sent = translation_vi_en(orig_sent)

In [ ]:
# Load T5-paraphraser
def set_seed(seed):
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

set_seed(42)


def en_paraphrase(sentence):
    text =  "paraphrase: " + sentence + " </s>"

    max_len = 256

    encoding = para_tokenizer.encode_plus(text,pad_to_max_length=True, return_tensors="pt")
    input_ids, attention_masks = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)


    # set top_k = 50 and set top_p = 0.95 and num_return_sequences = 3
    beam_outputs = para_model.generate(
        input_ids=input_ids, attention_mask=attention_masks,
        do_sample=True,
        max_length=256,
        top_k=120,
        top_p=0.98,
        early_stopping=True,
        num_return_sequences=10
    )


    # print ("\nOriginal Question ::")
    # print (sentence)
    # print ("\n")
    # print ("Paraphrased Questions :: ")
    final_outputs =[]
    for beam_output in beam_outputs:
        sent = para_tokenizer.decode(beam_output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
        if sent.lower() != sentence.lower() and sent not in final_outputs:
            final_outputs.append(sent)

    # for i, final_output in enumerate(final_outputs):
    #     print("{}: {}".format(i, final_output))

    return final_outputs

In [ ]:
# en_para_list = en_paraphrase(vi_en_sent)
# en_para_list

In [ ]:
# Dịch lại sang câu tiếng việt
def translation_en_vi(sentence_list):
    output_list = []
    for sentence in sentence_list:
        # print("check: ", sentence)
        tokenized_text = en_vi_tokenizer.encode(sentence, return_tensors="pt").to(device)
        en_vi_model.eval()
        summary_ids = en_vi_model.generate(
                            tokenized_text,
                            max_length=128, 
                            num_beams=1,
                            repetition_penalty=2.5, 
                            length_penalty=1.0, 
                            early_stopping=True
                        )
        output = en_vi_tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        # print(output)
        output_list.append(output)
    return output_list

In [ ]:
# en_vi_list = translation_en_vi(en_para_list)
# en_vi_list

In [ ]:
def generate_paraphrase(sentence):
    en_sent = translation_vi_en(sentence)
    en_para_lst = en_paraphrase(en_sent)
    vi_sent_lst = translation_en_vi(en_para_lst)
    return vi_sent_lst

In [ ]:
import time
start_time = time.time()
vi_para = generate_paraphrase(orig_sent)
print(time.time() - start_time)
print("Câu gốc: ", orig_sent)
for i, sent in enumerate(vi_para):
    print(f"Câu paraphrased: {i}", sent)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2232: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:192: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."


40.95884561538696
Câu gốc:  Điều này cho phép bạn lựa chọn những công việc cần phải hoàn thành trong ngày hôm đó và loại bỏ các công việc có thể hoàn thành vào các ngày khác .
Câu paraphrased:  Bằng cách đặt ngày và thời gian, bạn có thể chọn một công việc hoặc xoá nó, bạn sẽ không được yêu cầu hoàn thành nhiệm vụ vào những ngày khác.
Câu paraphrased:  Điều này cho phép bạn chọn các công việc cần hoàn thành vào ngày hôm nay và loại bỏ các công việc có thể hoàn thành vào những ngày khác.
Câu paraphrased:  Liệu có thể làm cho bạn được lên lịch cho một ngày cụ thể mà không phải quay lại và cập nhật lịch của riêng bạn cho ngày đó?
Câu paraphrased:  Khi bạn nhập vào lịch nhiệm vụ được tạo ra vào ngày đó, bạn được yêu cầu đưa ra lựa chọn này và loại bỏ các công việc mà bạn không thể hoàn thành trong những ngày khác.
Câu paraphrased:  Các công việc bạn phải hoàn thành trong ngày đó là gì?
Câu paraphrased:  Bạn sẽ phải làm gì nếu tôi đang học cho một B.Sc hoặc B.Tech? Không vấn đề gì.
Câu para